## Introduction

In the last notebook, we introduced generalize linear models (GLMs) that can be used to analyze non-normal data, such as count data or percentages. In general, GLMs tend to be much more powerful than linear models. However, there are a few statistical assumptions that you should be aware of...

For this notebook, we will continue to use the glucosinolate data that we downloaded earlier.

### Load the glucosinolate data

In [ ]:
# rm(list=ls());
# open the glucosinolate file in R
# same file as before...
glucosinolateFileName <- "data/cmeyer_glucs2015/bmeyer_etal.txt";  
glucs <- read.table(glucosinolateFileName, header=T, sep="\t", as.is=T, stringsAsFactors=FALSE);  
glucs <- glucs[order(glucs[,"accession_id"]),];
glucs[,3:ncol(glucs)] <- round(glucs[,3:ncol(glucs)]);

In [ ]:
# what's in the working environment?
ls();

### What is the statistical model?

With this notebook, we will continue to discuss *Poisson* family analyses. Poisson models (and their derivatives) are used to analyze count data, which - as you will remember - are very common.

Examples:
  - The number of insects within several $1m^2$ quadrats
  - The number of ions that hit a detector in a time interval
  - The number of meteorites that hit the Earth each year

 
The glucosinolate data are also count data (i.e. the number of glucosinolate ions that hit the QQQ detector in a given time frame).

In [ ]:
str(glucs)

In [ ]:
head(glucs);

In [ ]:
# if lme4, ggplot2, and gridExtra aren't installed, install them...
if( !require("lme4" )){  
    install.packages("lme4");  
}

if( !require("ggplot2" )){  
    install.packages("ggplot2");  
}


Recall that we used linear and generalized linear mixed models (i.e. fitted and random effects) to model BLUPs. As an example with a Poisson GLM:

In [ ]:
# previously, we used the following code to determine the total 'effort' or offset
totalIonCounts <- rowSums( glucs[,-c(1,2)] );
cat("The range of ion counts is now:", range(totalIonCounts), "\n");

In [ ]:
# the 0 indicates that there are missing data; get rid of those samples:
dropouts <- which( totalIonCounts == 0 );
glucs2 <- glucs[-dropouts,];
dim(glucs2);

totalIonCounts <- rowSums( glucs2[,-c(1,2)] );
cat("The range of ion counts is now:", range(totalIonCounts), "\n");

In [ ]:
# we can now fit the mixed model:
glmer0 <- glmer( G2P ~ 1 + offset(log(totalIonCounts)) + (1|accession_id), data=glucs2, control=glmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)), family="poisson" );
glmBlups <- ranef(glmer0)$accession_id; # these are the best-linear unbiased predictors:
glmBlups <- data.frame( accession_id=rownames(glmBlups), pois_blup=glmBlups[,1], stringsAsFactors=FALSE );
tail(glmBlups);

### Model assumptions

The Poisson distribution is used with non-negative integers that have no natural upper bound. However, there are other assumptions, including independence among events. For example, the number of people buying lunch each minute is not really Poisson distributed, because (a) people go to lunch together and (b) people eat at certain times (among other factors).

Furthermore, the mean and variance of a Poisson random variable are expected to be equal and are described by the single parameter $\lambda$.  

When the mean and variance are *not* equal, then the data are either underdispersed or overdispersed. Underdispersion (mean > variance) is rare compared to overdispersion (mean < variance) and is often ignored.

<!--# however, these are count data (see the output from head above), which suggests 
# that a Poisson-family model should be used; let's try a simple quasi-Poisson GLM:
glm0 <- glm( G2P ~ 1, data=glucs, family="quasipoisson" );
glm0.res <- residuals( glm0 );
glm0.res[1:10];

# note: the names are no longer the accession Ids, but the row names from the glucs data frame. Do you know why?
# let's use a workaround:
################################################################################
## my version of stack, which avoids factor generation
################################################################################
mstack <- function(arg, newHeaders, setRowNames=T, sorted=TRUE, decreasing=F){
    values <- data.frame(names=I(names(arg)), values=as.numeric(arg));

    if( setRowNames ){
        rownames(values) <- values[,"names"];
    }
    
    if( sorted ) {
        values <- values[order(values[,"values"], decreasing=decreasing),];	
    }
    
    colnames(values) <- newHeaders;
    return(values);
}

glm0.res <- mstack( glm0.res, newHeaders=c("row_id", "residual"), sorted=FALSE );
head(glm0.res);-->

We can test for overdispersion using the [following code from Ben Bolker](http://bbolker.github.io/mixedmodels-misc/glmmFAQ.html#testing-for-overdispersioncomputing-overdispersion-factor):

In [ ]:
# a test for overdispersion, from Ben Bolker's GLMM website:
overdisp_fun <- function(model) {
    rdf <- df.residual(model)
    rp <- residuals(model,type="pearson")
    Pearson.chisq <- sum(rp^2)
    prat <- Pearson.chisq/rdf
    pval <- pchisq(Pearson.chisq, df=rdf, lower.tail=FALSE)
    c(chisq=Pearson.chisq,ratio=prat,rdf=rdf,p=pval)
}

In [ ]:
# determine whether the data are overdispersed:
glmer0 <- glmer( G2P ~ 1 + offset(log(totalIonCounts)) + (1|accession_id), data=glucs2, family="poisson" );
overdisp_fun( glmer0 );

The data are highly overdispersed.


In [ ]:
# one workaround is to simply update the coefficient table by multiplying the
# standard error by the square root of the dispersion factor (phi) and then 
# updating the Z-values and the corresponding P-values such as:

modelToTest <- coef(summary(glmer0));
phi <- overdisp_fun(glmer0)["ratio"];

modelToTest <- within(as.data.frame(modelToTest),
{   `Std. Error` <- `Std. Error`*sqrt(phi)
    `z value` <- Estimate/`Std. Error`
    `Pr(>|z|)` <- 2*pnorm(abs(`z value`), lower.tail=FALSE)
})

cat("Before:\n");
coef(summary(glmer0));

cat("\nAnd after:\n");
printCoefmat(modelToTest, digits=3);

In [ ]:
# another workaround: simply use glmer.nb as an alternative
# however glmer.nb is "slow and fragile" (see the Bolker wiki page noted above):
glmer.nb0 <- glmer.nb( G2P ~ 1 + offset(log(totalIonCounts)) + (1|accession_id), data=glucs2);

In [ ]:
# can we use quasi models with mixed models? Not with lme4. One alternative is to fit an olre
# or an 'observation level random effect'. This is somewhat conservative, but will aid you in accounting for overdispersion
glucs2 <- data.frame(olre=1:nrow(glucs2), glucs2, stringsAsFactors=FALSE);
glucs2[1:3,];

In [ ]:
# with the olre as a random effect:
glmer.olre <- glmer( G2P ~ 1 + offset(log(totalIonCounts)) + (1|olre) + (1|accession_id), data=glucs2, family="poisson");


### Last but not least, in a pinch, you could use residuals instead of BLUPs

In [ ]:
# another possible workaround
glm0 <- glm( G2P ~ 1, offset=log(totalIonCounts), data=glucs2, family="quasipoisson");
summary(glm0);

In [ ]:
# like BLUPs, residuals are regularly used in GWAS
glm0.res <- residuals( glm0 );
glm0.res[1:10];

In [ ]:
# note: the names are no longer the accession Ids, but the row names from the glucs data frame. Do you know why?
# let's use a workaround:
################################################################################
## my version of stack, which avoids factor generation
################################################################################
mstack <- function(arg, newHeaders, setRowNames=T, sorted=TRUE, decreasing=F){
    values <- data.frame(names=I(names(arg)), values=as.numeric(arg));

    if( setRowNames ){
        rownames(values) <- values[,"names"];
    }
    
    if( sorted ) {
        values <- values[order(values[,"values"], decreasing=decreasing),];	
    }
    
    colnames(values) <- newHeaders;
    return(values);
}

glm0.res <- mstack( glm0.res, newHeaders=c("row_id", "residual"), sorted=FALSE );
glucs2 <- data.frame( row_id=rownames(glucs2), glucs2, stringsAsFactors=FALSE);

# the residuals here only correspond to the metabolite we've been working with, so 
# to protect yourself from making downstream mistakes, subset to the metadata and that column
output <- merge(glm0.res, glucs2[,c("row_id", "accession_id", "sample_weight", "G2P")], by="row_id" );

In [ ]:
head(output);

<br><br>

One could then take the averages of each residual for each accession id, using an approach like the tapply approach described earlier.